In [ ]:

import importlib

if not importlib.util.find_spec("my_first_ltr"): # type: ignore
    %pip install -qqq git+https://github.com/algolia/my-first-learning-to-rank

# What's Learning To Rank ?

Ranking is the process of organizing or arranging items in order based on their importance, quality, or performance. It is widely used in various applications, such as search engines, where results are ranked to show the most relevant pages first, or in recommender systems, which rank products, movies, or other items to suggest the most suitable options to users.

## Why rank ? And what's ranking here ?
In the context of a search experience ranking helps prioritize the most relevant results, improving the user experience by making it easier to find what they need quickly. We will try to define an optimal order for the results for each query. What we define by query here is the word or phrase a user types to find information.

## What's ranking model ?

it's a function that maps an item to it's relevance score.

# It all starts with data

- a user search history on a streaming platform
- a subset of imdb dataset

At home, you can try with your search history if you'd like!
`# TODO: if so time, clean up the data prep script in a notebook`

We did all the nasty pre-processing and cleaning for you - so you can just have fun! 
The main steps we did for preprocessing are:
- One-hot encoding: turn multi-categorical features into a list of binary features
- Create a textual relevance signal (using [OkapiBM25](https://en.wikipedia.org/wiki/Okapi_BM25))
- Compute the relevance score of the documents.

## What are our features ?
What is a feature in our context? A feature in machine learning is a piece of information or characteristic that helps the model make predictions or decisions. It’s an input the model uses to learn patterns in the data.

In [ ]:
from my_first_ltr.utils import load_raw_dataset

# FIXME: remove for dataset
unprocess_dataset = load_raw_dataset()
unprocess_dataset.columns

Go ahead! Explore the dataset to get familiar with it a bit, here are a few examples for you!

In [ ]:
# histogram for numerical values:
unprocess_dataset.imdb_score.hist()

In [ ]:
# histogram for textual values (note the use of explode when it's a list of string):
unprocess_dataset.explode("genres").genres.value_counts().plot.barh()
unprocess_dataset[unprocess_dataset.Action == "play"].explode("genres").genres.value_counts().plot.barh(color="red")

In [ ]:
# correlation between values:
unprocess_dataset[["imdb_score", "tmdb_score"]].corr()

## What's our score here ?

- What's your idea ?

We base our scoring on the past interactions of the users with the movies when they typed a query. 
- We consider that if the user watched the movie, it was highly relevant
- We consider that if the user added it to it's watchlist, it was relevant, but not the right mood at that time
- We consider that if the user clicked on a movie, it showed some interest but it wasn't that relevant

In [ ]:
from my_first_ltr.utils import load_dataset

# TODO: check if once repo is public that loading works
dataset = load_dataset(local=True)
dataset.head(5)


# TODO: stuff we've learn and you should not waist time on


# TODO: show a specific query with results examples ?

We could also negative examples. It would help to improve the model by providing contrast to positive examples. Using a conversion ratio in the score, instead of a simple sum, helps the model better capture the relative importance of examples, leading to more balanced and accurate rankings.

# Then comes a model

## PointWise
## PairWise
## ListWise

We split data into training and testing sets to train the model on one portion of the data and evaluate its performance on unseen data, ensuring it generalizes well to new inputs.

In [ ]:
from my_first_ltr.train_utils import build_pool, dataset_split


train_df, test_df, val_df = dataset_split(dataset)

train_pool = build_pool(train_df, "train")
test_pool = build_pool(test_df, "test")
val_pool = build_pool(val_df, "validation")

## RMSE pointwise

We initializes a CatBoostRanker, a gradient-boosting model  using the following parameters:

- `loss_function="RMSE"`: Optimize based on `RMSE` which measures the average squared difference between predicted and true ranks.
- `learning_rate=0.15`: Determines how much the model's parameters are updated in response to the calculated error after each iteration. A smaller value leads to slower, more stable learning, while a larger value speeds learning but risks overshooting the optimal solution.
- `thread_count=1`: Uses a single CPU thread for training.
- `iterations=500`: Runs 500 iterations of boosting (adding weak learners to improve predictions).
- `random_seed=0`: Ensures reproducible results by fixing randomness.

In [ ]:

from catboost import CatBoostRanker

model = CatBoostRanker(loss_function="RMSE", depth=6, learning_rate=0.15, thread_count=1, iterations=500, random_seed=0)


model.fit(train_pool, eval_set=test_pool, plot=True, metric_period=1)



In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

val_df["pred_score"] = model.predict(val_pool)


results = val_df[["score", "pred_score"]]

print("Predictions vs Actuals:")
print(results.head())


rmse = np.sqrt(mean_squared_error(results["score"], results["pred_score"]))
mae = mean_absolute_error(results["score"], results["pred_score"])

print("\nEvaluation Metrics:")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")


## Try different approaches

In [ ]:
# FIXME: setup the ranker for some pairwise ranking
model_pairwise = CatBoostRanker(loss_function="PairLogit", thread_count=1)
model_pairwise.fit(train_pool, eval_set=test_pool, plot=True, metric_period=1)

In [ ]:
# FIXME: setup the ranker for some listwise ranking
# As you can see, we have to specify the end metric required to optimize here, (eg: CTR, CVR, NDCG) as we are not basing
# the optimization of the scores difference to prediction.
model_listwise = CatBoostRanker(loss_function="YetiRank", thread_count=1, custom_metric=["NDCG:top=-1;type=Base;denominator=LogPosition;hints=skip_train~false"])
model_listwise.fit(train_pool, eval_set=test_pool, plot=True, metric_period=1)

In [ ]:
from my_first_ltr.data_visualisation import RMSE, MAP, MAE, NDCG_20

# FIXME: Compare the different models. Little tip to get you started, you can use catboost `get_eval` method

your_metric = ...
model.eval_metrics(train_pool, your_metric, ntree_start=model.tree_count_ - 1)

In [ ]:
#@title Solution to retrieve metrics for multiple models

import pandas as pd

models = {"RMSE": model, "PairLogit": model_pairwise, "YetiRank": model_listwise}
metrics = []

for k, m in models.items():
    metrics_dict = dict()
    metrics_dict["model_name"] = k
    metrics_dict['train_NDCG@20'] = m.eval_metrics(train_pool,
                                                      'NDCG:top=20;type=Base;denominator=LogPosition',
                                                      ntree_start=m.tree_count_ - 1)['NDCG:top=20;type=Base'][0]

    metrics_dict['test_NDCG@20'] = m.eval_metrics(test_pool,
                                                     'NDCG:top=20;type=Base;denominator=LogPosition',
                                                     ntree_start=m.tree_count_ - 1)['NDCG:top=20;type=Base'][0]

    metrics_dict['val_NDCG@20'] = m.eval_metrics(val_pool,
                                                'NDCG:top=20;type=Base;denominator=LogPosition',
                                                ntree_start=m.tree_count_ - 1)['NDCG:top=20;type=Base'][0]
    metrics.append(metrics_dict)


metrics_df = pd.DataFrame.from_records(metrics)
metrics_df

## Feature importance

In [ ]:
import pandas as pd
from catboost import CatBoost, Pool
from my_first_ltr.train_utils import keep_input_features
import matplotlib.pyplot as plt

def cross_dataset_shape_cascade( m: CatBoost, X: pd.DataFrame, pools: dict[str, Pool]) -> None:
    """Shap values average per feature accross all dataset."""
    df_feature_importance = pd.DataFrame(
        data={k: m.get_feature_importance(pool) for k, pool in pools.items()}, index=X.columns
    )

    plt.close("all")
    df_feature_importance.plot.barh(figsize=(10, 12))
    plt.title("cross dataset - importance of each feature")
    plt.legend()
    plt.show()

    # plt.savefig(os.path.join(path, f"features_importances_{config.impl_name}.png"), format="png", bbox_inches="tight")

cross_dataset_shape_cascade(model, keep_input_features(train_df), {"train": train_pool, "test": test_pool, "val": val_pool})